In [1]:
#get json ds

##link to egfr_10_full_ft_pd_lines.json
#https://drive.google.com/file/d/1r_hC1sa1H2Ht3ZGlp-rkWbcN1a-CIr8I/view?usp=sharing

#!gdown --id "1r_hC1sa1H2Ht3ZGlp-rkWbcN1a-CIr8I"
!gdown --id "1pWwaJt8WSjBfZK5r-rxbO-svE43spPek"
#!gdown --id "1MVoKgsUFx5mdGNtZ7uSsCtygSOsrzu8u"
#!gdown --id "1W2gaP_8LdOjtkBKTxWItwwB2iNLxo2Fx"
path2data_full = '/content/hiv1_hcv.csv'

Downloading...
From: https://drive.google.com/uc?id=1pWwaJt8WSjBfZK5r-rxbO-svE43spPek
To: /content/hiv1_hcv.csv
5.44GB [00:51, 105MB/s] 


In [2]:
'''
use subset of the original hiv1_hcv if you don't have 20Gb RAM!
'''

import pandas as pd
subset = pd.read_csv('/content/hiv1_hcv.csv').sample(frac=0.50)
subset.to_csv('/content/subset_hiv1_hcv.csv', index=False)
del subset
path2data_subset = '/content/subset_hiv1_hcv.csv'

In [3]:
!pip install ray[default]
!pip install tensorboard_logger
!pip install tensorboardX
!pip install ConfigSpace
!pip install hpbandster

     |████████████████████████████████| 49.7MB 64kB/s 
     |████████████████████████████████| 1.0MB 47.4MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 3.1MB 48.5MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 133kB 38.1MB/s 
     |████████████████████████████████| 1.3MB 40.2MB/s 
     |████████████████████████████████| 204kB 48.5MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 143kB 39.8MB/s 
     |████████████████████████████████| 296kB 41.1MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-none-any.whl size=12621 sha256=e7df3aa84c03b280dbf8de19e7c10a9a9584585b8b9e9563a23ffa16cad526ca
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
Successfully built gpustat
  Found existing installation: protobuf 3.12.4
  

     |████████████████████████████████| 122kB 7.6MB/s 
     |████████████████████████████████| 952kB 8.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for ConfigSpace: filename=ConfigSpace-0.4.18-cp37-cp37m-linux_x86_64.whl size=2879772 sha256=cbfc4f376f17e553be395c48a7a76f9b012e9861d52578d32cbf9af76830d2ab
  Stored in directory: /root/.cache/pip/wheels/39/ea/40/d93931850f700427db0a84180829c709d30484c9475040c7bd
Successfully built ConfigSpace
     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 92kB 5.6MB/s 
  Created wheel for hpbandster: filename=hpbandster-0.7.4-cp37-none-any.whl size=80008 sha256=0b29a3c9fd350265fe616336c4bb66b2a2f2d2902e1458eb0a0eaf49f8479126
  Stored in directory: /root/.cache/pip/wheels/9d/57/62/6b00c8011bac96e0c404adc5be4e16964ba4544614240b4e23
  Created wheel for netifaces: filename=netifaces-0.10.9-cp37-cp37m-linux_x86_

In [4]:
import os
#if not os.path.isdir('checkpoint'):
    #!mkdir 'checkpoint'
checkpoint_dir = 'checkpoint/'

import copy
import warnings
import json
from filelock import FileLock

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import dataloader
import torch.optim as optim
from sklearn.metrics import precision_recall_curve
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler, HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
from functools import partial


from nets import UnitedNet
from dataset import ANYDataset, train_validation_split
from metrics import *
import utils

warnings.filterwarnings('ignore')

In [17]:
CONFIG_PARAM = ['lr', 'l2reg', 'dropout']
METRIC_PARAM=["train_loss", "train_sensitivity", "train_specificity", "train_accuracy", "train_mcc", "train_auc", 
              "val_loss", "val_sensitivity", "val_specificity", "val_accuracy", "val_mcc", "val_auc"]
              
#DATASET = '/content/hiv1_hcv.csv'
DATASET = '/content/subset_hiv1_hcv.csv'
#train_data, val_data = train_validation_split(DATASET)
#del train_data, val_data

NEPOCH = 500
BATCHSIZE = 128
HASHCODE = 'TEST'
USE_MAT = True
USE_MORD = True

METRICS_DICT = {
    'sensitivity': sensitivity,
    'specificity': specificity,
    'accuracy': accuracy,
    'mcc': mcc,
    'auc': auc,
}

if torch.cuda.is_available():
    TRAIN_DEVICE = 'cuda'
    VAL_DEVICE = 'cuda'
else:
    TRAIN_DEVICE = 'cpu'
    VAL_DEVICE = 'cpu'

In [18]:
config = {
    'lr': tune.loguniform(5e-6, 1e-4),
    'l2reg': tune.loguniform(5e-6, 1),
    'dropout': tune.uniform(0.0, 0.7),
}

In [19]:
def UnitedNet_wraper(config, dense_dim, smiles_len, use_mord=True, use_mat=True, infer=False, dir_path=None):
    return UnitedNet(dense_dim, smiles_len, use_mord=use_mord, use_mat=use_mat, infer=infer, dir_path=dir_path, 
                 vis_thresh=0.2, dropout=config['dropout'])

*****SINGLE RUN HT*****

In [20]:
def run_epoch(net, opt, criterion, train_loader, val_loader):

    metrics = dict((metric, []) for metric in METRIC_PARAM)

    # Train step
    train_losses = []
    train_outputs = []
    train_labels = []
    net.train()
    for i, (mord_ft, non_mord_ft, label) in enumerate(train_loader):
        mord_ft = mord_ft.float().to(TRAIN_DEVICE)
        non_mord_ft = non_mord_ft.view((-1, 1, int(non_mord_ft.shape[1]/42), 42)).float().to(TRAIN_DEVICE)
        mat_ft = non_mord_ft.squeeze(1).float().to(TRAIN_DEVICE)
        label = label.float().to(TRAIN_DEVICE)

        # Forward
        opt.zero_grad()

        outputs = net(non_mord_ft, mord_ft, mat_ft)
        outputs = torch.squeeze(outputs)
        
        loss = criterion(outputs, label)
        train_losses.append(float(loss.item()))
        train_outputs.extend(outputs)
        train_labels.extend(label)

        # Parameters update
        loss.backward()
        opt.step()
    
    # Validation step
    val_losses = []
    val_outputs = []
    val_labels = []
    net.eval()
    for i, (mord_ft, non_mord_ft, label) in enumerate(val_loader):
        mord_ft = mord_ft.float().to(VAL_DEVICE)
        non_mord_ft = non_mord_ft.view((-1, 1, int(non_mord_ft.shape[1]/42), 42)).float().to(VAL_DEVICE)
        mat_ft = non_mord_ft.squeeze(1).float().to(VAL_DEVICE)
        label = label.float().to(VAL_DEVICE)

        with torch.no_grad():
            outputs = net(non_mord_ft, mord_ft, mat_ft)
            outputs = torch.squeeze(outputs)
            
            loss = criterion(outputs, label)
            val_losses.append(float(loss.item()))
            val_outputs.extend(outputs)
            val_labels.extend(label)

    # Calculate metrics
    train_outputs = torch.stack(train_outputs)
    val_outputs = torch.stack(val_outputs)
    train_labels = torch.stack(train_labels)
    val_labels = torch.stack(val_labels)

    metrics['train_loss'] = sum(train_losses) / len(train_losses)
    metrics['val_loss'] = sum(val_losses) / len(val_losses)
    for key in METRICS_DICT.keys():
        metrics['train_' + key] = METRICS_DICT[key](train_labels, train_outputs)
        metrics['val_' + key] = METRICS_DICT[key](val_labels, val_outputs)

    for metric, measurements in metrics.items():
        metrics[metric] = np.mean(metrics[metric])

    return metrics

In [21]:
def single_run_tune(config, checkpoint_dir, nepoch, opt_type='adam', dense_dim=None, train_loader=None, val_loader=None):

    def make_loaders(dataset, batch_size):
        train_data, val_data = train_validation_split(dataset)
        train_dataset = ANYDataset(train_data)
        del train_data
        train_size = len(train_dataset)
        dense_dim = train_dataset.get_dim('mord')
        train_loader = dataloader.DataLoader(dataset=train_dataset, batch_size=batch_size, 
                                                collate_fn=utils.custom_collate, shuffle=True)
        del train_dataset

        val_dataset = ANYDataset(val_data)
        del val_data
        val_size = len(val_dataset)
        val_loader = dataloader.DataLoader(dataset=val_dataset,batch_size=batch_size,
                                            collate_fn=utils.custom_collate, shuffle=False)
        del val_dataset
        print('Dataloaders created: train {}, val {}'.format(train_size, val_size))
        return train_loader, val_loader, dense_dim

    lock = FileLock(DATASET + ".lock")
    with lock:
        train_loader, val_loader, dense_dim = make_loaders(DATASET, BATCHSIZE)

    mord_ft, non_mord_ft, label = next(iter(train_loader))
    smiles_len = int(non_mord_ft.shape[1]/42)

    criterion = nn.BCELoss()

    net = UnitedNet_wraper(config, dense_dim, smiles_len=smiles_len, 
                           use_mord=USE_MORD, use_mat=USE_MAT, infer=False, dir_path=None).to(TRAIN_DEVICE)
 
    if opt_type == 'sgd':
        opt = optim.SGD(net.parameters(), lr=config['lr'], momentum=0.99)
    elif opt_type == 'adam':
        opt = optim.Adam(net.parameters(), lr=config['lr'], weight_decay=config['l2reg'])

    '''
    if checkpoint_dir:
        path = os.path.join(checkpoint_dir, "checkpoint")
        try:
            model_state, optimizer_state = torch.load(path)
            net.load_state_dict(model_state)
            opt.load_state_dict(optimizer_state)
        except:
            print('Cant load the model')
    '''

    start_epoch = 0
    if checkpoint_dir:
        with open(os.path.join(checkpoint_dir, "checkpoint")) as f:
            state = json.loads(f.read())
            start = state["step"] + 1


    for epoch in range(start_epoch, nepoch):
        metrics = run_epoch(net, opt, criterion, train_loader, val_loader)
        
        '''
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), opt.state_dict()), path)
        '''

        with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            with open(path, "w") as f:
                f.write(json.dumps({"step": epoch}))

        tune.report(
            train_loss=metrics['train_loss'], 
            train_sensitivity=metrics['train_sensitivity'],
            train_specificity=metrics['train_specificity'],
            train_accuracy=metrics['train_accuracy'],
            train_mcc=metrics['train_mcc'],
            train_auc=metrics['train_auc'],
            val_loss=metrics['val_loss'],
            val_sensitivity=metrics['val_sensitivity'],
            val_specificity=metrics['val_specificity'],
            val_accuracy=metrics['val_accuracy'],
            val_mcc=metrics['val_mcc'],
            val_auc=metrics['val_auc']
        )  


In [22]:
def main(config, checkpoint_dir=None, num_samples=4, reduction_factor=4, max_concurrent=4, max_num_epochs=30,
         train_loader=None, val_loader=None, dense_dim=None):

    target_metric = 'val_mcc'
    target_direction = 'max'
    gpus_per_trial = torch.cuda.device_count() / max_concurrent
    gpus_per_trial = int(gpus_per_trial) if gpus_per_trial > 1 else gpus_per_trial
    cpu_per_trial = os.cpu_count() / max_concurrent
    cpu_per_trial = int(cpu_per_trial) if cpu_per_trial > 1 else cpu_per_trial
    print('gpus_per_trial:', gpus_per_trial, "cpu_per_trial:", cpu_per_trial)

    scheduler = HyperBandForBOHB(        
        time_attr="training_iteration",
        max_t=max_num_epochs,
        reduction_factor=reduction_factor,
        metric=target_metric,
        mode=target_direction
    )

    search_alg = TuneBOHB(
        # space=config_space,  # If you want to set the space manually
        max_concurrent=max_concurrent,
        metric=target_metric, 
        mode=target_direction
    )

    reporter = CLIReporter(
        parameter_columns=CONFIG_PARAM,
        metric_columns=["training_iteration", METRIC_PARAM[0]] + METRIC_PARAM[6:])

    result = tune.run(
        partial(single_run_tune,
                checkpoint_dir=checkpoint_dir,
                nepoch=max_num_epochs, 
                opt_type='adam',
                dense_dim=dense_dim,
                train_loader=train_loader,
                val_loader=val_loader),
        name="bohb_ht",
        resources_per_trial={"cpu": cpu_per_trial, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        search_alg=search_alg,
        progress_reporter=reporter,
        verbose=2,
        stop={"training_iteration": 128})

    best_trial = result.get_best_trial("val_loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["val_loss"]))

    return result



In [23]:
result = main(config, checkpoint_dir, num_samples=12, max_concurrent=1, reduction_factor=2, max_num_epochs=512)

2021-05-27 16:43:11,533	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-27 16:43:11,537	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


gpus_per_trial: 1.0 cpu_per_trial: 4
== Status ==
Memory usage on this node: 1.4/25.5 GiB
Using HyperBand: num_stopped=0 total_brackets=1
Round #0:
  Bracket(Max Size (n)=512, Milestone (r)=1, completed=0.0%): {PENDING: 1} 
Resources requested: 0/4 CPUs, 0/1 GPUs, 0.0/15.0 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/bohb_ht
Number of trials: 1/12 (1 PENDING)


(pid=2875) Dataloaders created: train 43576, val 10894
Trial DEFAULT_9fbe2430 reported train_loss=0.4849131705124707,train_sensitivity=0.818483251147764,train_specificity=0.5160621110203055,train_accuracy=0.5976913897558289,train_mcc=0.30035023718983705,train_auc=0.7631629679962495,val_loss=0.43622653602167616,val_sensitivity=0.8184647302904564,val_specificity=0.6383404148962759,val_accuracy=0.6861575178997613,val_mcc=0.40366898088306896,val_auc=0.8268477270378117,should_checkpoint=True with parameters={'lr': 5.7004358787090395e-05, 'l2reg': 0.04399803705226463, 'dropout': 0.1701

(pid=3306) 2021-05-27 17:10:47,148	INFO trainable.py:378 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/bohb_ht/DEFAULT_9fbe2430_1_dropout=0.17015,l2reg=0.043998,lr=5.7004e-05_2021-05-27_16-43-11/checkpoint_tmp0ef4b4/./
(pid=3306) 2021-05-27 17:10:47,149	INFO trainable.py:385 -- Current state after restoring: {'_iteration': 1, '_timesteps_total': None, '_time_total': 130.2701232433319, '_episodes_total': None}


(pid=3306) Dataloaders created: train 43576, val 10894
Trial DEFAULT_9fbe2430 reported train_loss=0.49326291287050217,train_sensitivity=0.8205237204557048,train_specificity=0.4955051235305212,train_accuracy=0.5832338902147971,train_mcc=0.2852189734072239,train_auc=0.7521807285902702,val_loss=0.4479506497466287,val_sensitivity=0.9304979253112033,val_specificity=0.3810297425643589,val_accuracy=0.5268955388287131,val_mcc=0.30067202469210147,val_auc=0.8289059521081003,should_checkpoint=True with parameters={'lr': 5.7004358787090395e-05, 'l2reg': 0.04399803705226463, 'dropout': 0.17015250752170036}.
== Status ==
Memory usage on this node: 10.7/25.5 GiB
Using HyperBand: num_stopped=0 total_brackets=1
Round #0:
  Bracket(Max Size (n)=256, Milestone (r)=2, completed=0.3%): {PENDING: 11, RUNNING: 1} 
Resources requested: 4.0/4 CPUs, 1.0/1 GPUs, 0.0/15.0 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/bohb_ht
Number of trials: 12/12 (11 PENDING, 1 R

(pid=3335) 2021-05-27 17:13:09,750	INFO trainable.py:378 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/bohb_ht/DEFAULT_eec28792_2_dropout=0.16393,l2reg=0.021477,lr=1.8964e-05_2021-05-27_16-45-24/checkpoint_tmpd93025/./
(pid=3335) 2021-05-27 17:13:09,750	INFO trainable.py:385 -- Current state after restoring: {'_iteration': 1, '_timesteps_total': None, '_time_total': 128.4598388671875, '_episodes_total': None}


...

(pid=5322) Dataloaders created: train 43576, val 10894
Trial DEFAULT_dd49d4c8 reported train_loss=0.4990296824411912,train_sensitivity=0.8103213739160007,train_specificity=0.4910731124662098,train_accuracy=0.5772443546906554,train_mcc=0.27206012730214196,train_auc=0.7426507725085063,val_loss=0.4622784343569778,val_sensitivity=0.9263485477178424,val_specificity=0.3560359910022494,val_accuracy=0.5074352854782449,val_mcc=0.2773961310733474,val_auc=0.8110420959766974,should_checkpoint=True with parameters={'lr': 4.443899569806201e-05, 'l2reg': 0.000556023739381709, 'dropout': 0.24974078735412877}.
== Status ==
Memory usage on this node: 10.8/25.5 GiB
Using HyperBand: num_stopped=8 total_brackets=1
Round #0:
  Bracket(Max Size (n)=4, Milestone (r)=128, completed=15.0%): {PENDING: 1, RUNNING: 1, TERMINATED: 10} 
Resources requested: 4.0/4 CPUs, 1.0/1 GPUs, 0.0/15.0 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/bohb_ht
Number of trials: 12/12 (

(pid=5373) 2021-05-28 02:08:15,758	INFO trainable.py:378 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/bohb_ht/DEFAULT_d285d6e4_11_dropout=0.34659,l2reg=0.069777,lr=4.1774e-05_2021-05-27_17-06-05/checkpoint_tmp981561/./
(pid=5373) 2021-05-28 02:08:15,759	INFO trainable.py:385 -- Current state after restoring: {'_iteration': 64, '_timesteps_total': None, '_time_total': 2624.2101078033447, '_episodes_total': None}


(pid=5373) Dataloaders created: train 43576, val 10894
Trial DEFAULT_d285d6e4 reported train_loss=0.5235627485859778,train_sensitivity=0.8015643598027546,train_specificity=0.42698183189790656,train_accuracy=0.5280888562511474,train_mcc=0.21070220403907483,train_auc=0.7009827634090675,val_loss=0.5013974333225295,val_sensitivity=0.9944674965421854,val_specificity=0.03299175206198451,val_accuracy=0.28823205434183957,val_mcc=0.0766246510116625,val_auc=0.7774621005882691,should_checkpoint=True with parameters={'lr': 4.1773540163428196e-05, 'l2reg': 0.06977716505389436, 'dropout': 0.34658896937368167}.
== Status ==
Memory usage on this node: 10.8/25.5 GiB
Using HyperBand: num_stopped=8 total_brackets=1
Round #0:
  Bracket(Max Size (n)=4, Milestone (r)=128, completed=16.2%): {RUNNING: 1, TERMINATED: 11} 
Resources requested: 4.0/4 CPUs, 1.0/1 GPUs, 0.0/15.0 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/bohb_ht
Number of trials: 12/12 (1 RUNNING

2021-05-28 02:39:33,393	INFO tune.py:549 -- Total run time: 35781.86 seconds (35774.73 seconds for the tuning loop).


Best trial config: {'lr': 2.6864513559255806e-05, 'l2reg': 7.433448344750201e-05, 'dropout': 0.3611129739986707}
Best trial final validation loss: 0.31850305927354233


In [26]:
import pandas as pd

future_df = {}
for k, v in result.results.items():
    observ = {}
    for conf in CONFIG_PARAM:
        observ[conf] = result.results[k]['config'][conf]
    for metric in METRIC_PARAM + ['training_iteration']:
        observ[metric] = result.results[k][metric]
    future_df[k] = observ

df = pd.DataFrame.from_dict(future_df, orient='index')
df = df.sort_values('val_mcc', ascending=False)
df

,lr,l2reg,dropout,train_loss,train_sensitivity,train_specificity,train_accuracy,train_mcc,train_auc,val_loss,val_sensitivity,val_specificity,val_accuracy,val_mcc,val_auc,training_iteration
dd49d4c8,0.000044,0.000556,0.249741,0.051535,0.985207,0.969542,0.973770,0.935797,0.997925,0.483101,0.746542,0.906148,0.863778,0.651402,0.894626,128
eec28792,0.000019,0.021477,0.163935,0.049893,0.988182,0.972245,0.976547,0.942506,0.998306,0.447210,0.741010,0.902524,0.859648,0.641356,0.886254,128
d285d6e4,0.000042,0.069777,0.346589,0.192141,0.945758,0.905388,0.916284,0.807118,0.977721,0.340132,0.768326,0.866783,0.840646,0.610946,0.877645,128
40fa1bce,0.000057,0.012363,0.081431,0.059512,0.985207,0.971208,0.974986,0.938613,0.997576,0.429771,0.764523,0.864909,0.838260,0.605276,0.876755,128
9fbe2430,0.000057,0.043998,0.170153,0.148471,0.944907,0.914597,0.922779,0.819717,0.983221,0.352469,0.788036,0.843539,0.828805,0.595864,0.890270,64
23c0ba9c,0.000070,0.000175,0.661016,0.258704,0.872131,0.832118,0.842918,0.652119,0.937267,0.320454,0.801176,0.822794,0.817055,0.580484,0.896443,64
810441ac,0.000080,0.070236,0.080160,0.193623,0.932579,0.897592,0.907036,0.785895,0.973378,0.345562,0.765560,0.845789,0.824491,0.580420,0.879690,64
e5da2cf6,0.000027,0.000074,0.361113,0.227755,0.896106,0.849092,0.861782,0.692629,0.952225,0.318503,0.824343,0.794801,0.802644,0.566420,0.897612,64
9370a224,0.000010,0.940119,0.073157,0.298475,0.897551,0.777079,0.809597,0.610070,0.931191,0.343992,0.846819,0.737441,0.766477,0.523139,0.886413,32
8ad1c28c,0.000018,0.029831,0.429220,0.325946,0.840078,0.776859,0.793923,0.561583,0.897655,0.344128,0.872061,0.677331,0.729025,0.486167,0.885543,32


In [27]:
file_name = 'bohb_ht_Data1half_12_base_512_mcc'
df.to_csv(file_name + '.csv')

In [28]:
df_small = df[CONFIG_PARAM + ['training_iteration', 'train_loss', "val_loss", "val_sensitivity", "val_specificity", "val_accuracy", "val_mcc", "val_auc"]]
df_small.to_csv(file_name + '_small.csv')
df_small.head(3)

,lr,l2reg,dropout,training_iteration,train_loss,val_loss,val_sensitivity,val_specificity,val_accuracy,val_mcc,val_auc
dd49d4c8,0.000044,0.000556,0.249741,128,0.051535,0.483101,0.746542,0.906148,0.863778,0.651402,0.894626
eec28792,0.000019,0.021477,0.163935,128,0.049893,0.447210,0.741010,0.902524,0.859648,0.641356,0.886254
d285d6e4,0.000042,0.069777,0.346589,128,0.192141,0.340132,0.768326,0.866783,0.840646,0.610946,0.877645
